#AIPI 590 XAI Adversarial Patch

In [1]:
## Standard libraries
import os
import json
import math
import time
import numpy as np
import scipy.linalg

## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms

# Use GPU if possible
device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:0")
print("Using device", device)

# Root Folder
ROOT = os.path.dirname(os.path.abspath(__file__))
DATA_PATH = os.path.join(ROOT, "data")
MODEL_PATH = os.path.join(ROOT, "model")

<ipython-input-1-9bb6122fd041>:13: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export
INFO:lightning_fabric.utilities.seed:Seed set to 42


Using device cuda:0


In [ ]:
## Download the small image dataset for training
import urllib.request
from urllib.error import HTTPError
import zipfile
base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial10/"
os.makedirs(DATA_PATH, exist_ok=True)

file_path = os.path.join(DATA_PATH, "TinyImageNet.zip")
if not os.path.isfile(file_path):
    file_url = base_url + "TinyImageNet.zip"
    try:
        urllib.request.urlretrieve(file_url, file_path)
    except HTTPError as e:
        print("Error downloading data", e)
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(file_path.rsplit("/",1)[0])


In [3]:
# Load CNN architecture pretrained on ImageNet
pretrained_model = torchvision.models.resnet34(weights='IMAGENET1K_V1')
pretrained_model = pretrained_model.to(device)

pretrained_model.eval()
for p in pretrained_model.parameters():
    p.requires_grad = False

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to ../saved_models/tutorial10/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 221MB/s]


In [4]:
# Mean and Std from ImageNet
NORM_MEAN = np.array([0.485, 0.456, 0.406])
NORM_STD = np.array([0.229, 0.224, 0.225])

# Image tensor
image_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=NORM_MEAN,
                         std=NORM_STD)
])

# Load dataset
dataset = torchvision.datasets.ImageFolder(root=os.path.join(DATA_PATH, "TinyImageNet"),
                                           transform=image_transforms)

# Load label
with open(os.path.join(DATA_PATH, "imagenet_class_index.json"), "r") as f:
    label_names = json.load(f)

In [62]:
## Apply patch to image (each will add two patches with 32 x 32 inches)
TENSOR_MEANS, TENSOR_STD = torch.FloatTensor(NORM_MEAN)[:,None,None], torch.FloatTensor(NORM_STD)[:,None,None]
def apply_patch(img, patch, num_patch=2):
    patch = (torch.tanh(patch) + 1 - 2 * TENSOR_MEANS) / (2 * TENSOR_STD)
    for i in range(img.shape[0]):
        for _ in range(num_patch):
          h_offset = np.random.randint(0,img.shape[2]-patch.shape[1]-1)
          w_offset = np.random.randint(0,img.shape[3]-patch.shape[2]-1)
          img[i,:,h_offset:h_offset+patch.shape[1],w_offset:w_offset+patch.shape[2]] = patch
    return img

In [54]:
## Train patch towards the target label
def train_patch(model, target_class, num_epochs=5):
    train_loader = data.DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True, num_workers=8)

    patch = nn.Parameter(torch.zeros(3, 32, 32), requires_grad=True)
    optimizer = torch.optim.SGD([patch], lr=1e-1, momentum=0.8)
    loss_module = nn.CrossEntropyLoss()

    # Training loop
    for epoch in range(num_epochs):
        t = tqdm(train_loader, leave=False)
        for img, _ in t:
            img = apply_patch(img, patch)
            img = img.to(device)
            pred = model(img)
            labels = torch.zeros(img.shape[0], device=pred.device, dtype=torch.long).fill_(target_class)
            loss = loss_module(pred, labels)
            optimizer.zero_grad()
            loss.mean().backward()
            optimizer.step()
            t.set_description(f"Epoch {epoch}, Loss: {loss.item():4.2f}")

    return patch.data

In [43]:
# Get patch from pre-trained record or retrain a new one.
def get_patch(class_name):
    c = label_names.index(class_name)
    file_name = os.path.join(MODEL_PATH, f"{class_name}_patch.pt")
    if not os.path.isfile(file_name):
        patch = train_patch(pretrained_model, c)
        torch.save(patch, file_name)
    else:
        patch = torch.load(file_name)
    return patch

In [63]:
# This is the actual patch used.
patch = get_patch('flamingo')

  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

Validating...:   0%|          | 0/16 [00:00<?, ?it/s]

Validation results for flamingo: {'acc': 0.983433723449707, 'top5': 0.9974899888038635}


### References

**Duke AI XAI adversarial-ai-example-notebooks:**

https://github.com/AIPI-590-XAI/Duke-AI-XAI/blob/main/adversarial-ai-example-notebooks/adversarial_attacks.ipynb